In [1]:
import json
import math
import re
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from tqdm.auto import tqdm


In [2]:
def calculate_sharpe(pnl_series, periods_per_year=390 * 252):
    std = pnl_series.std()
    if std == 0:
        return 0.0
    raw = pnl_series.mean() / std
    return raw * np.sqrt(periods_per_year)


## Load data

In [3]:
DATA_PATH = Path('final_df.csv')
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Cannot find {DATA_PATH}")

df_ori = pd.read_csv(DATA_PATH)
df = df_ori.copy()
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(['symbol', 'datetime']).reset_index(drop=True)

df['lret_1m'] = df.groupby('symbol')['close'].transform(lambda s: np.log(s).diff())
df['y_target'] = df.groupby('symbol')['lret_1m'].shift(-1)

initial_rows = len(df)
df = df[df['y_target'].abs() <= 0.2].dropna(subset=['y_target', 'lret_1m'])
df = df.sort_values(['datetime', 'symbol']).reset_index(drop=True)
print(f"Dropped {initial_rows - len(df)} rows with abnormal/NaN targets")
print(df[['datetime', 'symbol', 'close', 'lret_1m', 'y_target']].head())


Dropped 2 rows with abnormal/NaN targets
             datetime symbol     close   lret_1m  y_target
0 2024-04-30 12:51:00   AMAT  200.5100 -0.000199 -0.000100
1 2024-04-30 12:52:00   AMAT  200.4900 -0.000100  0.000149
2 2024-04-30 12:53:00   AMAT  200.5198  0.000149  0.000549
3 2024-04-30 12:54:00   AMAT  200.6300  0.000549 -0.000523
4 2024-04-30 12:55:00   AMAT  200.5250 -0.000523  0.000175


## Feature cleaning

In [4]:
y = df['y_target']
X = df.drop(columns=[
    'y_target', 'lret_1m', 'datetime', 'symbol',
    'year', 'month', 'day', 'minute', 'minute_of_day'
], errors='ignore')
print(f"Original feature shape: {X.shape}")

stats = X.describe(percentiles=[0.99]).T
bad_cols = set()
bad_cols.update(stats.index[stats['std'] > 1e3])
bad_cols.update(stats.index[stats['99%'].abs() > 1e3])
bad_cols.update(stats.index[stats['max'].abs() > 1e6])
bad_cols.update(stats.index[stats['std'] == 0])

bad_cols = sorted(bad_cols)
print(f"Dropping {len(bad_cols)} problematic columns")
X_cleaned = X.drop(columns=bad_cols)
print(f"Cleaned feature shape: {X_cleaned.shape}")


Original feature shape: (19111, 184)
Dropping 53 problematic columns
Cleaned feature shape: (19111, 131)


## Train / test split

In [5]:
split_ratio = 1.0 / 1.5
split_index = int(len(X_cleaned) * split_ratio)

X_train = X_cleaned.iloc[:split_index]
y_train = y.iloc[:split_index]
X_test = X_cleaned.iloc[split_index:]
y_test = y.iloc[split_index:]

print(f"Train shape: {X_train.shape}")
print(f"Test shape:  {X_test.shape}")


Train shape: (12740, 131)
Test shape:  (6371, 131)


## Preprocessing pipeline

Same as in LASSO, ridge regression notebook

In [6]:
class QuantileClipper(BaseEstimator, TransformerMixin):
    def __init__(self, lower=0.005, upper=0.995):
        self.lower = lower
        self.upper = upper
        self.q_low_ = None
        self.q_high_ = None

    def fit(self, X, y=None):
        self.q_low_ = np.nanpercentile(X, self.lower * 100, axis=0)
        self.q_high_ = np.nanpercentile(X, self.upper * 100, axis=0)
        return self

    def transform(self, X):
        return np.clip(X, self.q_low_, self.q_high_)

feature_pipeline = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('clip', QuantileClipper(lower=0.005, upper=0.995)),
    ('scale', MaxAbsScaler()),
])

feature_pipeline.fit(X_train)

X_all_processed = pd.DataFrame(
    feature_pipeline.transform(X_cleaned),
    columns=X_cleaned.columns,
    index=X_cleaned.index
)

print("Preprocessing complete")


Preprocessing complete


In [7]:
feature_cols = X_cleaned.columns.tolist()
df_processed = X_all_processed.copy()
df_processed[['datetime', 'symbol', 'y_target']] = df[['datetime', 'symbol', 'y_target']].values

train_df = df_processed.iloc[:split_index].copy()
test_df = df_processed.iloc[split_index:].copy()


## Sequence construction

In [8]:
SEQ_LEN = 30
PRED_HORIZON = 1


def build_sequences(dataframe, feature_cols, target_col, seq_len=SEQ_LEN):
    X_seq, y_seq, meta_dt, meta_symbol = [], [], [], []
    for symbol, group in dataframe.groupby('symbol'):
        group = group.sort_values('datetime')
        feats = group[feature_cols].values
        target = group[target_col].values
        dts = group['datetime'].values
        for i in range(len(group) - seq_len - PRED_HORIZON + 1):
            start = i
            end = i + seq_len
            target_idx = end + PRED_HORIZON - 1
            X_seq.append(feats[start:end])
            y_seq.append(target[target_idx])
            meta_dt.append(dts[target_idx])
            meta_symbol.append(symbol)
    return (
        np.array(X_seq, dtype=np.float32),
        np.array(y_seq, dtype=np.float32),
        np.array(meta_dt),
        np.array(meta_symbol)
    )

train_X_seq, train_y_seq, train_meta_dt, train_meta_sym = build_sequences(
    train_df, feature_cols, 'y_target', seq_len=SEQ_LEN
)

test_X_seq, test_y_seq, test_meta_dt, test_meta_sym = build_sequences(
    test_df, feature_cols, 'y_target', seq_len=SEQ_LEN
)

print(f"Train sequences: {train_X_seq.shape}")
print(f"Test sequences:  {test_X_seq.shape}")


Train sequences: (12710, 30, 131)
Test sequences:  (6341, 30, 131)


In [9]:
train_order = np.argsort(train_meta_dt)
train_X_seq = train_X_seq[train_order]
train_y_seq = train_y_seq[train_order]
train_meta_dt = train_meta_dt[train_order]

test_order = np.argsort(test_meta_dt)
test_X_seq = test_X_seq[test_order]
test_y_seq = test_y_seq[test_order]
test_meta_dt = test_meta_dt[test_order]


In [10]:
y_scaler = StandardScaler()
train_y_seq_raw = train_y_seq.copy()
test_y_seq_raw = test_y_seq.copy()

train_y_seq = y_scaler.fit_transform(train_y_seq.reshape(-1, 1)).astype(np.float32).ravel()
test_y_seq = y_scaler.transform(test_y_seq.reshape(-1, 1)).astype(np.float32).ravel()


## Prefix K-fold DataLoaders

In [11]:
BATCH_SIZE = 256

def create_prefix_folds(X_seq, y_seq, n_splits=10, batch_size=BATCH_SIZE):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    folds = []
    for fold_idx, (train_idx, val_idx) in enumerate(tscv.split(X_seq), start=1):
        X_tr, y_tr = X_seq[train_idx], y_seq[train_idx]
        X_va, y_va = X_seq[val_idx], y_seq[val_idx]
        train_ds = TensorDataset(torch.from_numpy(X_tr), torch.from_numpy(y_tr))
        val_ds = TensorDataset(torch.from_numpy(X_va), torch.from_numpy(y_va))
        folds.append((
            DataLoader(train_ds, batch_size=batch_size, shuffle=True),
            DataLoader(val_ds, batch_size=batch_size, shuffle=False)
        ))
        print(f"Fold {fold_idx}: train {len(train_ds)} seq, val {len(val_ds)} seq")
    return folds

prefix_folds = create_prefix_folds(train_X_seq, train_y_seq, n_splits=10)


Fold 1: train 1160 seq, val 1155 seq
Fold 2: train 2315 seq, val 1155 seq
Fold 3: train 3470 seq, val 1155 seq
Fold 4: train 4625 seq, val 1155 seq
Fold 5: train 5780 seq, val 1155 seq
Fold 6: train 6935 seq, val 1155 seq
Fold 7: train 8090 seq, val 1155 seq
Fold 8: train 9245 seq, val 1155 seq
Fold 9: train 10400 seq, val 1155 seq
Fold 10: train 11555 seq, val 1155 seq


## Transformer model

In [12]:
class ReturnTransformer(nn.Module):
    def __init__(self, feature_dim, d_model=64, nhead=4, num_layers=2, ff_dim=128, dropout=0.3):
        super().__init__()
        self.input_proj = nn.Linear(feature_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True,
            activation='gelu'
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.head = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_proj(x)
        encoded = self.encoder(x)
        return self.head(encoded[:, -1, :]).squeeze(-1)


## Training utilities

In [13]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    for xb, yb in loader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item() * len(xb)
    return total_loss / len(loader.dataset)


def eval_epoch(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            pred = model(xb)
            loss = criterion(pred, yb)
            total_loss += loss.item() * len(xb)
    return total_loss / len(loader.dataset)


def predict_batches(model, X_array, batch_size=512):
    model.eval()
    preds = []
    with torch.no_grad():
        for start in range(0, len(X_array), batch_size):
            batch = torch.from_numpy(X_array[start:start + batch_size]).to(DEVICE)
            preds.append(model(batch).cpu().numpy())
    return np.concatenate(preds)


In [14]:
def evaluate_sharpe(model, loader, scaler):
    model.eval()
    preds_list, rets_list = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            preds = model(xb).cpu().numpy()
            rets = yb.cpu().numpy()
            preds_raw = scaler.inverse_transform(preds.reshape(-1, 1)).ravel()
            rets_raw = scaler.inverse_transform(rets.reshape(-1, 1)).ravel()
            preds_list.append(preds_raw)
            rets_list.append(rets_raw)
    preds = np.concatenate(preds_list)
    rets = np.concatenate(rets_list)
    return calculate_sharpe(pd.Series(preds * rets))


## Prefix CV training

In [15]:
import copy

def fit_with_prefix_cv(folds, feature_dim, epochs=15, lr=1e-3, weight_decay=1e-4, patience=3):
    histories = []
    best_states = []
    for fold_idx, (train_loader, val_loader) in enumerate(folds, start=1):
        model = ReturnTransformer(feature_dim).to(DEVICE)
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.MSELoss()
        best_val = math.inf
        best_sharpe = -math.inf
        best_state = None
        patience_ctr = 0
        for epoch in range(1, epochs + 1):
            train_loss = train_epoch(model, train_loader, optimizer, criterion)
            val_loss = eval_epoch(model, val_loader, criterion)
            val_sharpe = evaluate_sharpe(model, val_loader, y_scaler)
            improved = False
            if val_loss < best_val - 1e-6:
                improved = True
            elif abs(val_loss - best_val) <= 1e-6 and val_sharpe > best_sharpe + 1e-3:
                improved = True
            if improved:
                best_val = val_loss
                best_sharpe = val_sharpe
                best_state = copy.deepcopy(model.state_dict())
                patience_ctr = 0
            else:
                patience_ctr += 1
            print(f"Fold {fold_idx} | Epoch {epoch} | train {train_loss:.6f} | val {val_loss:.6f} | val_SR {val_sharpe:.4f}")
            if patience_ctr >= patience:
                print(f"Stopping early on fold {fold_idx}")
                break
        histories.append({'fold': fold_idx,
                          'best_val_mse': best_val,
                          'best_val_sharpe': best_sharpe})
        best_states.append(best_state)
    return histories, best_states

cv_histories, cv_states = fit_with_prefix_cv(
    prefix_folds,
    feature_dim=len(feature_cols),
    epochs=12,
    lr=1e-3,
    weight_decay=1e-4,
    patience=3,
)
cv_histories


Fold 1 | Epoch 1 | train 0.821110 | val 0.361741 | val_SR -12.9365
Fold 1 | Epoch 2 | train 0.653966 | val 0.250345 | val_SR 7.2762
Fold 1 | Epoch 3 | train 0.613870 | val 0.243173 | val_SR 1.4122
Fold 1 | Epoch 4 | train 0.594202 | val 0.215090 | val_SR 8.6539
Fold 1 | Epoch 5 | train 0.574316 | val 0.214358 | val_SR 5.6651
Fold 1 | Epoch 6 | train 0.572633 | val 0.237596 | val_SR 7.6779
Fold 1 | Epoch 7 | train 0.563306 | val 0.218192 | val_SR 3.9724
Fold 1 | Epoch 8 | train 0.563268 | val 0.216904 | val_SR 6.1427
Stopping early on fold 1
Fold 2 | Epoch 1 | train 0.652810 | val 0.537185 | val_SR -8.3652
Fold 2 | Epoch 2 | train 0.435574 | val 0.480148 | val_SR 0.5324
Fold 2 | Epoch 3 | train 0.406145 | val 0.479036 | val_SR 10.4968
Fold 2 | Epoch 4 | train 0.401675 | val 0.474253 | val_SR 21.6939
Fold 2 | Epoch 5 | train 0.397393 | val 0.480376 | val_SR -5.0319
Fold 2 | Epoch 6 | train 0.391339 | val 0.483561 | val_SR 5.4250
Fold 2 | Epoch 7 | train 0.394291 | val 0.484104 | val_SR 8

[{'fold': 1,
  'best_val_mse': 0.21435808161378422,
  'best_val_sharpe': np.float64(5.665060368494481)},
 {'fold': 2,
  'best_val_mse': 0.4742531318620686,
  'best_val_sharpe': np.float64(21.693905478080694)},
 {'fold': 3,
  'best_val_mse': 0.8693194981777307,
  'best_val_sharpe': np.float64(-13.857241924483475)},
 {'fold': 4,
  'best_val_mse': 0.32517106115043937,
  'best_val_sharpe': np.float64(16.562425174177765)},
 {'fold': 5,
  'best_val_mse': 0.4584722115234895,
  'best_val_sharpe': np.float64(-16.8689710876445)},
 {'fold': 6,
  'best_val_mse': 0.1258370710948071,
  'best_val_sharpe': np.float64(20.204365899362813)},
 {'fold': 7,
  'best_val_mse': 1.8681512706207506,
  'best_val_sharpe': np.float64(1.354979909058905)},
 {'fold': 8,
  'best_val_mse': 1.2127228436789987,
  'best_val_sharpe': np.float64(10.159013603135019)},
 {'fold': 9,
  'best_val_mse': 3.4878782105910315,
  'best_val_sharpe': np.float64(3.6951565440633796)},
 {'fold': 10,
  'best_val_mse': 1.4163185643943357,
  '

## Final training on full training window

In [16]:
VALID_SPLIT = int(len(train_X_seq) * 0.9)
full_train_ds = TensorDataset(
    torch.from_numpy(train_X_seq[:VALID_SPLIT]),
    torch.from_numpy(train_y_seq[:VALID_SPLIT])
)
full_val_ds = TensorDataset(
    torch.from_numpy(train_X_seq[VALID_SPLIT:]),
    torch.from_numpy(train_y_seq[VALID_SPLIT:])
)
full_train_loader = DataLoader(full_train_ds, batch_size=BATCH_SIZE, shuffle=True)
full_val_loader = DataLoader(full_val_ds, batch_size=BATCH_SIZE, shuffle=False)

final_model = ReturnTransformer(len(feature_cols)).to(DEVICE)
optimizer = torch.optim.AdamW(final_model.parameters(), lr=1e-3, weight_decay=1e-3)
criterion = nn.MSELoss()

best_state = None
best_val = math.inf
best_sharpe = -math.inf
patience = 4
patience_ctr = 0
divergent_ctr = 0

for epoch in range(1, 21):
    train_loss = train_epoch(final_model, full_train_loader, optimizer, criterion)
    val_loss = eval_epoch(final_model, full_val_loader, criterion)
    val_sharpe = evaluate_sharpe(final_model, full_val_loader, y_scaler)

    improved = False
    if val_loss < best_val - 1e-6:
        improved = True
    elif abs(val_loss - best_val) <= 1e-6 and val_sharpe > best_sharpe + 1e-3:
        improved = True

    if improved:
        best_val = val_loss
        best_sharpe = val_sharpe
        best_state = copy.deepcopy(final_model.state_dict())
        patience_ctr = 0
        divergent_ctr = 0
    else:
        patience_ctr += 1
        if val_sharpe < 0:
            divergent_ctr += 1

    print(f"Epoch {epoch} | train {train_loss:.6f} | val {val_loss:.6f} | val_SR {val_sharpe:.4f}")

    if patience_ctr >= patience or divergent_ctr >= 2:
        print("Early stopping triggered (loss plateau / Sharpe deterioration).")
        break

final_model.load_state_dict(best_state)
print(f"Best val MSE: {best_val:.6f}, best val Sharpe: {best_sharpe:.4f}")


Epoch 1 | train 1.000361 | val 1.334176 | val_SR -16.5330
Epoch 2 | train 0.978309 | val 1.335929 | val_SR -17.1215
Epoch 3 | train 0.970826 | val 1.336135 | val_SR -15.1484
Early stopping triggered (loss plateau / Sharpe deterioration).
Best val MSE: 1.334176, best val Sharpe: -16.5330


## Sharpe evaluation

In [17]:
train_preds = predict_batches(final_model, train_X_seq)
train_preds_raw = y_scaler.inverse_transform(train_preds.reshape(-1, 1)).ravel()
train_pnl = pd.Series(train_preds_raw * train_y_seq_raw)
train_sharpe = calculate_sharpe(train_pnl)

test_preds = predict_batches(final_model, test_X_seq)
test_preds_raw = y_scaler.inverse_transform(test_preds.reshape(-1, 1)).ravel()
test_pnl = pd.Series(test_preds_raw * test_y_seq_raw)
test_sharpe = calculate_sharpe(test_pnl)

print(f"Train Sharpe: {train_sharpe:.6f}")
print(f"Test Sharpe:  {test_sharpe:.6f}")


Train Sharpe: 6.402199
Test Sharpe:  3.562124


## Rolling 30-minute backtest (optional heavy cell)

In [29]:
ROLLING_WINDOW_MINUTES = 60
MIN_SEQ_PER_WINDOW = 20


def build_window_sequences(window_df, feature_cols, seq_len=SEQ_LEN):
    X_seq, y_seq = [], []
    for symbol, group in window_df.groupby('symbol'):
        group = group.sort_values('datetime')
        if len(group) < seq_len + 1:
            continue
        feats = group[feature_cols].values
        target = group['y_target'].values
        for i in range(len(group) - seq_len):
            X_seq.append(feats[i:i+seq_len])
            y_seq.append(target[i+seq_len])
    if not X_seq:
        return None, None
    return np.array(X_seq, dtype=np.float32), np.array(y_seq, dtype=np.float32)

backtest_results = []
unique_test_minutes = np.sort(test_df['datetime'].unique())

for current_dt in tqdm(unique_test_minutes, desc='Rolling backtest'):
    start_dt = current_dt - pd.Timedelta(minutes=ROLLING_WINDOW_MINUTES)
    window_df = df_processed[(df_processed['datetime'] >= start_dt) & (df_processed['datetime'] < current_dt)]
    X_tr, y_tr = build_window_sequences(window_df, feature_cols, seq_len=SEQ_LEN)
    if X_tr is None or len(X_tr) < MIN_SEQ_PER_WINDOW:
        continue
    window_scaler = StandardScaler()
    y_tr_scaled = window_scaler.fit_transform(y_tr.reshape(-1, 1)).astype(np.float32).ravel()
    train_loader = DataLoader(
        TensorDataset(torch.from_numpy(X_tr), torch.from_numpy(y_tr_scaled)),
        batch_size=128,
        shuffle=True
    )
    temp_model = ReturnTransformer(len(feature_cols)).to(DEVICE)
    optimizer = torch.optim.AdamW(temp_model.parameters(), lr=1e-3, weight_decay=1e-4)
    criterion = nn.MSELoss()
    for _ in range(3):
        train_epoch(temp_model, train_loader, optimizer, criterion)

    # Build prediction sequences ending at current_dt for each symbol
    pred_rows = df_processed[df_processed['datetime'] == current_dt]
    symbol_preds = []
    combined = pd.concat([window_df, pred_rows], ignore_index=True)
    for symbol, group in combined.groupby('symbol'):
        group = group.sort_values('datetime')
        if group['datetime'].iloc[-1] != current_dt:
            continue
        if len(group) < SEQ_LEN:
            continue
        seq = group[feature_cols].values[-SEQ_LEN:]
        symbol_preds.append((symbol, seq))

    if not symbol_preds:
        continue

    batch = np.stack([seq for _, seq in symbol_preds]).astype(np.float32)
    batch = torch.from_numpy(batch).to(DEVICE)
    temp_model.eval()
    with torch.no_grad():
        preds_scaled = temp_model(batch).cpu().numpy()
    preds_raw = window_scaler.inverse_transform(preds_scaled.reshape(-1, 1)).ravel()


    actuals = df_processed[df_processed['datetime'] == current_dt][['symbol', 'y_target']]
    actual_map = dict(zip(actuals['symbol'], actuals['y_target']))
    for (symbol, _), pred in zip(symbol_preds, preds_raw):
        if symbol not in actual_map:
            continue
        backtest_results.append({
            'datetime': current_dt,
            'symbol': symbol,
            'predicted_log_return': pred,
            'actual_log_return': actual_map[symbol]
        })

backtest_df = pd.DataFrame(backtest_results)
if not backtest_df.empty:
    backtest_df['positive_prediction'] = backtest_df['predicted_log_return'].clip(lower=0)
    minute_sum = backtest_df.groupby('datetime')['positive_prediction'].transform('sum')
    backtest_df['weight_relative'] = np.where(minute_sum == 0, 0.0, backtest_df['positive_prediction'] / minute_sum)
    backtest_df.drop(columns=['positive_prediction'], inplace=True)
    backtest_df['weight_sign'] = np.sign(backtest_df['predicted_log_return'])
    print(backtest_df.head())
    print(f"Backtest rows: {len(backtest_df)}")
else:
    print("Backtest skipped due to insufficient sequences")


Rolling backtest:   0%|          | 0/6371 [00:00<?, ?it/s]

             datetime symbol  predicted_log_return  actual_log_return  \
0 2024-08-15 13:40:00   AMAT              0.000154           0.002123   
1 2024-08-15 13:41:00   AMAT              0.000481           0.000448   
2 2024-08-15 13:42:00   AMAT             -0.000249          -0.000848   
3 2024-08-15 13:43:00   AMAT              0.000362          -0.000330   
4 2024-08-15 13:44:00   AMAT              0.000298           0.001414   

   weight_relative  weight_sign  
0              1.0          1.0  
1              1.0          1.0  
2              0.0         -1.0  
3              1.0          1.0  
4              1.0          1.0  
Backtest rows: 4682


In [30]:
backtest_df.head(5)

,datetime,symbol,predicted_log_return,actual_log_return,weight_relative,weight_sign
0,2024-08-15 13:40:00,AMAT,0.000154,0.002123,1.0,1.0
1,2024-08-15 13:41:00,AMAT,0.000481,0.000448,1.0,1.0
2,2024-08-15 13:42:00,AMAT,-0.000249,-0.000848,0.0,-1.0
3,2024-08-15 13:43:00,AMAT,0.000362,-0.000330,1.0,1.0
4,2024-08-15 13:44:00,AMAT,0.000298,0.001414,1.0,1.0


In [31]:
pnl_series = backtest_df["weight_sign"] * backtest_df["actual_log_return"]
# 或 long-only 策略：backtest_df["weight_relative"] * actual_log_return

raw_sharpe = pnl_series.mean() / pnl_series.std()

# 年化：根据频率设定一年多少个观测点
minutes_per_day = 390        # 美股常规盘
trading_days_per_year = 252
periods_per_year = minutes_per_day * trading_days_per_year

annual_sharpe = raw_sharpe * np.sqrt(periods_per_year)
print(raw_sharpe, annual_sharpe)


-0.01704421911954488 -5.3433015301615185


In [32]:
# save backtest results
BACKTEST_PATH = Path('transformer_backtest_results.csv')
backtest_df.to_csv(BACKTEST_PATH, index=False)
print(f"Backtest results saved to {BACKTEST_PATH}")

Backtest results saved to transformer_backtest_results.csv
